# Adding a new metric

In this notebook, we will add a simple new metric to the system. Specifically, we will use sklearn's F1 score.

We start with the usual imports, and add scikit-learn to the list:

In [ ]:
# We'll suppress warnings because both altair and sklearn are
# emitting lots of them, and they're annoying in a demo setting.

import warnings
warnings.filterwarnings("ignore")

import altair as alt
# Ask Altair to produce output that works on Jupyter Notebook
alt.renderers.enable('notebook')

import fairness
import fairness.benchmark

import sklearn.metrics

In [ ]:
from fairness.metrics.Metric import Metric

class F1Score(Metric):
    def __init__(self):
        Metric.__init__(self)
        self.name = 'f1score'
    def calc(self, actual, predicted, *args):
        return sklearn.metrics.f1_score(actual, predicted)

fairness.add_metric(F1Score())

In [ ]:
fairness.benchmark.run(algorithm=["Feldman-DecisionTree", "DecisionTree"], dataset=["ricci"])

In [ ]:
ricci_Race = fairness.get_dataset_by_name("ricci").get_results_data_frame("Race", "numerical-binsensitive")

In [ ]:
list(ricci_Race.columns.values)

In [ ]:
alt.Chart(ricci_Race).mark_point().encode(
    x='accuracy',
    y='f1score',
    color='algorithm'
)

## Parameterized metrics

We provide some parameterized metrics, like `SensitiveMetric`, which takes a subordinate metric as a parameter and returns the average of the metric result conditioned on the sensitive attribute. For example:

In [ ]:
from fairness.metrics.SensitiveMetric import SensitiveMetric

fairness.add_metric(SensitiveMetric(F1Score))

In [ ]:
fairness.benchmark.run(algorithm=["Feldman-DecisionTree", "DecisionTree"], dataset=["ricci"])

In [ ]:
ricci_Race = fairness.get_dataset_by_name("ricci").get_results_data_frame("Race", "numerical-binsensitive")

Note the presence of the columns `Race-f1score`, `Race-f1scoreDiff`, and `Race-f1scoreRatio`:

In [ ]:
list(ricci_Race.columns.values)

In [ ]:
ricci_Race["Race_f1score"] = ricci_Race["Race-f1score"] # work around bug in plotting library
ricci_Race["Race_f1scoreDiff"] = ricci_Race["Race-f1scoreDiff"] # work around bug in plotting library
alt.Chart(ricci_Race).mark_point().encode(
    x='accuracy',
    y='Race_f1scoreDiff',
    color='algorithm'
)